In [ ]:
import nibabel as nib
import matplotlib.pyplot as plt
import numpy as np
import cv2
from scipy.ndimage import zoom, rotate
from tqdm import tqdm
import os
from os.path import join, dirname, basename, exists
from pprint import pprint
import pickle
import pandas as pd

In [ ]:
dataset_path = '../../spreadsheet/Clinical_Metadata_FDG_PET_CT_Lesions.csv'


In [ ]:
df = pd.read_excel(dataset_path)

for row_idx in tqdm(df.index):
    ori_path = df.loc[row_idx, '...']  #path to the raw SUV data
    data = nib.load(ori_path)
    img = data.get_fdata()

    x,y,z = img.shape
    x_scale, y_scale, z_scale = data.header['pixdim'][1:4]
    min_scale = min(x_scale, y_scale, z_scale)
    x_scale /= min_scale
    y_scale /= min_scale
    z_scale /= min_scale

    seg_path = df.loc[row_idx, '...']  #path to the reconstructed segmentation
    with open(seg_path, 'rb') as f:
        seg = pickle.load(f)
        
    revert_seg = zoom(seg, (1.0/x_scale, 1.0/y_scale, 1.0/z_scale), order=0)  
    #Spline order 0 is equivalent to nearest neighbor interpolation.

    #write revert_seg to the path you want
            